https://github.com/ibatra/BERT-Keyword-Extractor/blob/master/BERT-Keyword%20Extractor.ipynb

In [2]:
import os
from nltk.tokenize import sent_tokenize

In [3]:
pwd

'/Users/kimkwangil/Project/01EssayFitAI/KeywordExtractor'

In [4]:
os.listdir()

['KeywordExtractionWithBert_pytorch.ipynb',
 'maui-semeval2010-train',
 'maui-semeval2010-test',
 'BERT-Keyword Extractor.ipynb',
 '.ipynb_checkpoints']

In [5]:
#train_path = "essay/01_Harvard"
train_path = "maui-semeval2010-train"

.txt로 끝나는 문서를 모두 txt로 바꿔준다.
.key 파일도 모두 key로 바꿔준다.

In [6]:
txt = sorted([f for f in os.listdir(train_path) if not f.endswith("-justTitle.txt") and not f.endswith(".key") and not f.endswith("-CrowdCountskey")])
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")])

In [7]:
filekey = dict()
for i, k in enumerate(txt):
    filekey[key[i]] = k

In [8]:
filekey

{'C-41.key': 'C-41.txt',
 'C-42.key': 'C-42.txt',
 'C-44.key': 'C-44.txt',
 'C-45.key': 'C-45.txt',
 'C-46.key': 'C-46.txt',
 'C-48.key': 'C-48.txt',
 'C-49.key': 'C-49.txt',
 'C-50.key': 'C-50.txt',
 'C-52.key': 'C-52.txt',
 'C-53.key': 'C-53.txt',
 'C-54.key': 'C-54.txt',
 'C-55.key': 'C-55.txt',
 'C-56.key': 'C-56.txt',
 'C-57.key': 'C-57.txt',
 'C-58.key': 'C-58.txt',
 'C-61.key': 'C-61.txt',
 'C-62.key': 'C-62.txt',
 'C-65.key': 'C-65.txt',
 'C-66.key': 'C-66.txt',
 'C-67.key': 'C-67.txt',
 'C-68.key': 'C-68.txt',
 'C-69.key': 'C-69.txt',
 'C-71.key': 'C-71.txt',
 'C-72.key': 'C-72.txt',
 'C-74.key': 'C-74.txt',
 'C-75.key': 'C-75.txt',
 'C-76.key': 'C-76.txt',
 'C-77.key': 'C-77.txt',
 'C-78.key': 'C-78.txt',
 'C-79.key': 'C-79.txt',
 'C-80.key': 'C-80.txt',
 'C-81.key': 'C-81.txt',
 'C-83.key': 'C-83.txt',
 'C-84.key': 'C-84.txt',
 'H-35.key': 'H-35.txt',
 'H-37.key': 'H-37.txt',
 'H-38.key': 'H-38.txt',
 'H-40.key': 'H-40.txt',
 'H-41.key': 'H-41.txt',
 'H-42.key': 'H-42.txt',


In [9]:
def convert(key):
    sentences = ""
    for line in open(train_path + "/" + filekey[key], 'r', encoding='utf8' ):
        sentences += (" " + line.rstrip())
    tokens = sent_tokenize(sentences)
    key_file = open(train_path + "/" + str(key),'r')
    keys = [line.strip() for line in key_file]
    key_sent = []
    labels = []
    for token in tokens:
        z = ['O'] * len(token.split())
        for k in keys:
            if k in token:
                
                if len(k.split())==1:
                    try:
                        z[token.lower().split().index(k.lower().split()[0])] = 'B'
                    except ValueError:
                        continue
                elif len(k.split())>1:
                    try:
                        if token.lower().split().index(k.lower().split()[0]) and token.lower().split().index(k.lower().split()[-1]):
                            z[token.lower().split().index(k.lower().split()[0])] = 'B'
                            for j in range(1, len(k.split())):
                                z[token.lower().split().index(k.lower().split()[j])] = 'I'
                    except ValueError:
                        continue
        for m, n in enumerate(z):
            if z[m] == 'I' and z[m-1] == 'O':
                z[m] = 'O'

        if set(z) != {'O'}:
            labels.append(z) 
            key_sent.append(token)
    return key_sent, labels

In [10]:
sentences_ = []
labels_ = []
for key, value in filekey.items():
    s, l = convert(key)
    sentences_.append(s)
    labels_.append(l)
sentences = [item for sublist in sentences_ for item in sublist]
labels = [item for sublist in labels_ for item in sublist]
# print(len(sentences), len(labels))

Model

In [11]:
import torch

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [17]:
MAX_LEN = 75  #문자를 읽는 양
bs = 32 #배치사이즈

In [18]:
tag2idx = {'B': 0, 'I': 1, 'O': 2} #태그를 인덱스로 만들고 키앖을 정함 0,1,2
tags_vals = ['B', 'I', 'O'] #태그값을 리스트로 만듬

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gpu를 사용하겠다는거. 있다면ㅎ 나는 있으니까 사용!
n_gpu = torch.cuda.device_count()

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # 버트 사전훈련한 모델을 토큰으로 만듬

100%|██████████| 231508/231508 [00:04<00:00, 50617.68B/s]


In [21]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences] #토큰화된 텍스트를 리스트로 담음

In [22]:
#ids, tags를 불러와서 패딩작업 처리함.... 여전히 전처리중...
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (766 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (714 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (544 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (630 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1001 > 512). Running this sequence through BERT will result in indexing errors


In [23]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids] #마스크처리해서 가려줌. 맞춰보랑게. 마스크를

In [24]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [25]:
#traing, validation을  input, tags, masks로 구분해서리 가각 텐서로 만들어서

tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [26]:
#데이터로 로딩해서리

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [27]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx)) #분류모델을 가져와서

100%|██████████| 407873900/407873900 [2:30:04<00:00, 45295.35B/s]  


In [30]:
model = model.cpu() #쿠다에 때려넣겠다고 함수를 선언한 다음에

In [36]:
#완전 파인튜닝 했다고하고, 손실없어, 최적화하겠다고 함

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [38]:
!pip install seqeval[cpu]

  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=57ebfd47ec43b534c080d2617d091b78d428e36405cd3afa1f8dadfb007123a2
  Stored in directory: /Users/kimkwangil/Library/Caches/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


In [39]:
from seqeval.metrics import f1_score #f1스코어를 비교하자 이제. 이게 잘 나오면 성능으 좋은거지

def flat_accuracy(preds, labels): #이건 정확도를 측정하는 함수넹. 
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
epochs = 1
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

In [ ]:
def keywordextract(sentence):
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

In [ ]:
text = "The solution is based upon an abstract representation of the mobile object system."

In [ ]:
keywordextract(text)